# Example expected syntax of the R client
Not expected to be functional yet

In [1]:
library(devtools)
options(unzip = 'internal'); install_github("Open-EO/openeo-r-backend", ref="develop") # Currently takes a while
install_github("Open-EO/openeo-r-client", ref="develop")
library(openeo)

Skipping install of 'openEO.R.Backend' from a github remote, the SHA1 (ff026e0a) has not changed since last install.
  Use `force = TRUE` to force installation
Skipping install of 'openeo' from a github remote, the SHA1 (c9b21546) has not changed since last install.
  Use `force = TRUE` to force installation
Loading required package: magrittr
Loading required package: jsonlite


(Potentially there should be an ability to explore available/connected backends)

## Connect to a backend

In [2]:
conn = connect("http://localhost:8000/api", "test", "test")

Login successful.

## Listing backend capabilities:
Get a list of functions that the backend natively supports; names and descriptions (list/named vector)

In [6]:
listProcesses(conn)

NULL

Same for listing collections on the backend

In [3]:
listCollections(conn)

[[1]]
[[1]]$product_id
[1] "landsat7_ndvi"

[[1]]$description
[1] "Landsat7 NDVI calculation"

[[1]]$source
[1] "Marius Appel"


[[2]]
[[2]]$product_id
[1] "sentinel2_subset"

[[2]]$description
[1] "Sentinel 2 raster time series for a small spatial subset"

[[2]]$source
[1] "ESA / Marius Appel"

Get in-depth information about a process or collection

In [4]:
describe(conn, "sentinel2_subset")

[[1]]
[[1]]$status
[1] 404

[[1]]$message
[1] "Process not found"

Listing existing user jobs

In [8]:
listJobs(conn)

NULL

Listing which UDF capabilities/packages are supported

In [9]:
listUDFCapabilities(conn)

ERROR: Error in listUDFCapabilities(conn): could not find function "listUDFCapabilities"


## Simple task: calculating NDVI using a linked list of processes
Assumes NDVI function takes band names as parameters

In [17]:
Task = collection("S2_L2A_T32TPS_20M") %>% process("filter_daterange", start="2016", end="2018") %>%
    process("NDVI", red="B04", nir="B8A")
str(Task)
taskToJSON(Task)

List of 4
 $ collections:List of 4
  ..$ collections:List of 1
  .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  ..$ process_id : chr "filter_daterange"
  ..$ start      : chr "2016"
  ..$ end        : chr "2018"
 $ process_id : chr "NDVI"
 $ red        : chr "B04"
 $ nir        : chr "B8A"


{
  "collections": {
    "collections": {
      "collection_id": "S2_L2A_T32TPS_20M"
    },
    "process_id": "filter_daterange",
    "start": "2016",
    "end": "2018"
  },
  "process_id": "NDVI",
  "red": "B04",
  "nir": "B8A"
} 

## Advanced task: calculating NDVI using processes that have other processes as arguments 
Assumes NDVI function takes process graphs

In [3]:
AOICollection = collection("S2_L2A_T32TPS_20M") %>% process("filter_daterange", start="2016", end="2018")
RedSubset = AOICollection %>% process("filter_band", band="red")
NIRSubset = AOICollection %>% process("filter_band", band="nir")
Task = AOICollection %>% process("NDVI", red=RedSubset, nir=NIRSubset)

str(Task)
taskToJSON(Task)

List of 4
 $ collections:List of 4
  ..$ collections:List of 1
  .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  ..$ process_id : chr "filter_daterange"
  ..$ start      : chr "2016"
  ..$ end        : chr "2018"
 $ process_id : chr "NDVI"
 $ red        :List of 3
  ..$ collections:List of 4
  .. ..$ collections:List of 1
  .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. ..$ process_id : chr "filter_daterange"
  .. ..$ start      : chr "2016"
  .. ..$ end        : chr "2018"
  ..$ process_id : chr "filter_band"
  ..$ band       : chr "red"
 $ nir        :List of 3
  ..$ collections:List of 4
  .. ..$ collections:List of 1
  .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. ..$ process_id : chr "filter_daterange"
  .. ..$ start      : chr "2016"
  .. ..$ end        : chr "2018"
  ..$ process_id : chr "filter_band"
  ..$ band       : chr "nir"


{
  "collections": {
    "collections": {
      "collection_id": "S2_L2A_T32TPS_20M"
    },
    "process_id": "filter_daterange",
    "start": "2016",
    "end": "2018"
  },
  "process_id": "NDVI",
  "red": {
    "collections": {
      "collections": {
        "collection_id": "S2_L2A_T32TPS_20M"
      },
      "process_id": "filter_daterange",
      "start": "2016",
      "end": "2018"
    },
    "process_id": "filter_band",
    "band": "red"
  },
  "nir": {
    "collections": {
      "collections": {
        "collection_id": "S2_L2A_T32TPS_20M"
      },
      "process_id": "filter_daterange",
      "start": "2016",
      "end": "2018"
    },
    "process_id": "filter_band",
    "band": "nir"
  }
} 

## Three ways of processing data in OpenEO:
### Synchronous
Run right away and give data as an object (Raster?); 'format' optional, should default to native that is then read into a Raster* object

In [ ]:
Result = executeTask(conn, Task, format="GTiff")

### Batch
Ask the server to prepare it the USGS ESPA way, get a job ID and the URL to the output location

In [ ]:
OutputInfo = orderTask(conn, Task)
JobID = OutputInfo$JobID

### Lazy
Ask the server to run it when needed (on WCS, but also download etc.)

In [ ]:
JobID = queueTask(conn, Task)

## Functions that can be performed with a JobID
Lazy and batch only

In [ ]:
followJob(conn, JobID) # cat --follow style updates about the job
JobInfo = queryJob(conn, JobID) # Get current information on a job
cancelJob(conn, JobID) # Pause a job; need a way to restart it also
deleteJob(conn, JobID) # Delete the job from the server entirely
Result = downloadJob(conn, JobID) # Get the result as with the synchronous case
(WCSURL = getWCSLink(conn, JobID)) # Get a URL to the WCS to visualise/download data

## Either way have a result, should be a **Raster*** object

In [ ]:
plot(Result)
spplot(Result, 3)

## Theoretical use case 1: composite example

In [19]:
CompositeTask = collection("S2_L2A_T32TPS_20M") %>% process("date_range_filter", start="2016-01-01", end="2016-03-10") %>%
    process("bbox_filter", left=652000, right=672000, top=5161000, bottom=5181000, srs="EPSG:32632") %>%
    process("max_time")

str(CompositeTask)
taskToJSON(CompositeTask)

List of 2
 $ collections:List of 7
  ..$ collections:List of 4
  .. ..$ collections:List of 1
  .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. ..$ process_id : chr "date_range_filter"
  .. ..$ start      : chr "2016-01-01"
  .. ..$ end        : chr "2016-03-10"
  ..$ process_id : chr "bbox_filter"
  ..$ left       : num 652000
  ..$ right      : num 672000
  ..$ top        : num 5161000
  ..$ bottom     : num 5181000
  ..$ srs        : chr "EPSG:32632"
 $ process_id : chr "max_time"


{
  "collections": {
    "collections": {
      "collections": {
        "collection_id": "S2_L2A_T32TPS_20M"
      },
      "process_id": "date_range_filter",
      "start": "2016-01-01",
      "end": "2016-03-10"
    },
    "process_id": "bbox_filter",
    "left": 652000,
    "right": 672000,
    "top": 5161000,
    "bottom": 5181000,
    "srs": "EPSG:32632"
  },
  "process_id": "max_time"
} 

### Lazy

In [ ]:
Conn = connect(host="http://saocompute.eurac.edu/openEO_WCPS_Driver", user="nobody", password="nobody")
Job = queueTask(Conn, CompositeTask)
OutPath = file.path("Downloads", "Result.netcdf")
downloadJob(Conn, Job, OutPath, "netcdf") # Processing happens here
Result = brick(OutPath)

### Synchronous

In [ ]:
Result = executeTask(Conn, CompositeTask)

In [ ]:
plot(Result)

## Complex use case: Land cover classification

UDF definitions: not in API yet!

In [ ]:
defineUDF(conn, udf_id="resample", type="aggregate_space", content=file.path("resample.r")) # This could be a server-defined function too
defineUDF(conn, udf_id="temporal_cloud_filter", type="apply_time", content=file.path("temporal_cloud_filter.r")) # No such type in API yet
defineUDF(conn, udf_id="harmonic_analysis", type="reduce_time", content=file.path("harmonic_analysis.r"))
defineUDF(conn, udf_id="ranger_classification", type="apply_pixel", content=file.path("ranger_classification.r"))

Process graphs

In [26]:
DEMCollection = collection("GLSDEM") %>%
    process("bbox_filter", left=652000, right=672000, top=5161000, bottom=5181000, srs="EPSG:32632") %>%
    process("resample", collection("S2_L2A_T32TPS_20M"))
    # UDF to resample to Sentinel 2 pixel size

CloudlessNDVI = collection("S2_L2A_T32TPS_20M") %>% # Graph part reused in several places
    process("bbox_filter", left=652000, right=672000, top=5161000, bottom=5181000, srs="EPSG:32632") %>%
    process("mask", mask="CM", maskvalue=5) %>% process("ndvi", red="B04", nir="B8A") %>%
    process("temporal_cloud_filter", band="B02", threshold="50")
    # Temporal cloud filter UDF: for each pixel use blue to detect outliers, replace more than 50 with NA

TSParams = CloudlessNDVI %>% process("harmonic_analysis")
# Harmonic parameter extraction UDF: get several layers of harmonic model params

MeanNDVI = CloudlessNDVI %>% process("mean_time")
# Also get the mean NDVI as a covariate

# Somehow upload training dataset or have a UDF that downloads it(?)
LandCoverTask = collection(NULL) %>% process("ranger_classification", NDVI=MeanNDVI, DEM=DEMCollection, TS=TSParams)
# UDF: runs a ranger training and prediction pass, returns an LC map and some (spatial?) statistics

str(LandCoverTask)
taskToJSON(LandCoverTask)

List of 5
 $ collections:List of 1
  ..$ collection_id: NULL
 $ process_id : chr "ranger_classification"
 $ NDVI       :List of 2
  ..$ collections:List of 4
  .. ..$ collections:List of 4
  .. .. ..$ collections:List of 4
  .. .. .. ..$ collections:List of 7
  .. .. .. .. ..$ collections:List of 1
  .. .. .. .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. .. .. .. ..$ process_id : chr "bbox_filter"
  .. .. .. .. ..$ left       : num 652000
  .. .. .. .. ..$ right      : num 672000
  .. .. .. .. ..$ top        : num 5161000
  .. .. .. .. ..$ bottom     : num 5181000
  .. .. .. .. ..$ srs        : chr "EPSG:32632"
  .. .. .. ..$ process_id : chr "mask"
  .. .. .. ..$ mask       : chr "CM"
  .. .. .. ..$ maskvalue  : num 5
  .. .. ..$ process_id : chr "ndvi"
  .. .. ..$ red        : chr "B04"
  .. .. ..$ nir        : chr "B8A"
  .. ..$ process_id : chr "temporal_cloud_filter"
  .. ..$ band       : chr "B02"
  .. ..$ threshold  : chr "50"
  ..$ process_id : chr "mean_time"
 $ DEM   

{
  "collections": {
    "collection_id": {}
  },
  "process_id": "ranger_classification",
  "NDVI": {
    "collections": {
      "collections": {
        "collections": {
          "collections": {
            "collections": {
              "collection_id": "S2_L2A_T32TPS_20M"
            },
            "process_id": "bbox_filter",
            "left": 652000,
            "right": 672000,
            "top": 5161000,
            "bottom": 5181000,
            "srs": "EPSG:32632"
          },
          "process_id": "mask",
          "mask": "CM",
          "maskvalue": 5
        },
        "process_id": "ndvi",
        "red": "B04",
        "nir": "B8A"
      },
      "process_id": "temporal_cloud_filter",
      "band": "B02",
      "threshold": "50"
    },
    "process_id": "mean_time"
  },
  "DEM": {
    "collections": {
      "collections": {
        "collection_id": "GLSDEM"
      },
      "process_id": "bbox_filter",
      "left": 652000,
      "right": 672000,
      "top": 5161000

In [ ]:
Result = executeTask(conn, LandCoverTask)
spplot(Result, 1)